# feats using polar yay

In [116]:
import polars as pl
import yaml
from pathlib import Path

# pl.set_option("display", False)

In [117]:
input_path = Path("../data/cleaned/2015_2024_statcast_clean.parquet")

df = pl.read_parquet(input_path)

In [118]:
df = df.sort(
    [
        "game_date",
        "game_pk",
        "at_bat_number",
        "pitch_number",
    ],
    descending=[False, False, False, True],
)


df.head()

df = df.with_columns(
    df.select(
        pl.col(pl.String).exclude(["player_name"]).cast(pl.Categorical).to_physical()
    ),
)

df
df = df.fill_null(-1)
df = df.sort(
    [
        "game_date",
        "game_pk",
        "at_bat_number",
        "pitch_number",
    ],
    descending=True,
)

df = df.with_columns(
    df.select(pl.col("pitch_type").shift(-1).alias("next_pitch")),
).drop_nulls("next_pitch")

df

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,zone,stand,p_throws,type,hit_location,bb_type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,bat_speed,swing_length,next_pitch
i64,datetime[μs],f64,f64,f64,str,i64,i64,i64,f64,i64,i64,i64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64
7,2024-10-20 00:00:00,78.8,4.1,5.27,"""Manaea, Sean""",571970,640455,4,14.0,1,0,2,-1.0,-1,3,1,-1.04,0.07,1.44,1.65,-1.0,-1.0,-1.0,0,3,0,-1.0,-1.0,-3.876776,-114.790473,-1.213129,-8.519711,20.865706,-31.531093,2.99,1.5,-1.0,-1.0,-1.0,79.7,2141.0,6.8,775302,53.72,-1.0,0.689131,0.7,1.0,0.0,0.0,-1.0,31,5,4,1,2,0,294.0,-1.0,-1.0,0
0,2024-10-20 00:00:00,83.9,4.16,5.13,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,2,1,1.28,-0.01,-1.13,1.32,-1.0,-1.0,-1.0,0,3,0,-1.0,-1.0,-14.910726,-121.300964,-2.306161,15.532788,23.944392,-32.030777,2.99,1.4,-1.0,-1.0,-1.0,84.7,1835.0,7.2,775302,53.31,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,4,4,1,2,0,106.0,-1.0,-1.0,1
1,2024-10-20 00:00:00,91.9,4.03,4.97,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,1,1,1.16,0.76,-0.93,2.17,-1.0,-1.0,-1.0,0,3,0,-1.0,-1.0,-15.210428,-132.944842,-2.837582,16.955867,27.981449,-22.734943,2.99,1.53,-1.0,-1.0,-1.0,92.7,2131.0,7.0,775302,53.48,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,3,4,1,2,0,113.0,-1.0,-1.0,7
7,2024-10-20 00:00:00,80.2,3.84,5.31,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,0,1,-1.34,0.15,-0.86,1.28,-1.0,-1.0,-1.0,0,3,0,-1.0,-1.0,-7.987162,-116.480556,-2.438514,-10.452965,23.959752,-30.532682,2.99,1.35,-1.0,-1.0,-1.0,81.3,2117.0,7.4,775302,53.14,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,2,4,1,2,0,284.0,-1.0,-1.0,7
7,2024-10-20 00:00:00,78.9,3.89,5.26,"""Manaea, Sean""",571970,640455,-1,8.0,1,0,1,-1.0,-1,0,0,-1.57,0.19,-0.23,1.53,-1.0,-1.0,-1.0,0,3,0,-1.0,-1.0,-6.125394,-114.782271,-1.669025,-12.671426,21.628193,-30.380521,2.99,1.48,-1.0,-1.0,-1.0,79.9,2152.0,7.0,775302,53.45,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,1,4,1,2,0,283.0,-1.0,-1.0,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,2024-03-16 00:00:00,84.0,1.71,6.4,"""Skubal, Tarik""",668804,669373,-1,14.0,0,0,1,-1.0,-1,1,1,1.52,0.6,1.09,2.08,-1.0,-1.0,457705.0,0,1,0,-1.0,-1.0,-4.432459,-122.07474,-4.703139,16.032842,26.558863,-25.453648,3.3,1.67,-1.0,-1.0,-1.0,83.5,1505.0,6.3,747947,54.23,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,3,0,0,0,1,130.0,-1.0,-1.0,0
0,2024-03-16 00:00:00,85.4,1.95,6.33,"""Skubal, Tarik""",668804,669373,-1,12.0,0,0,2,-1.0,-1,0,1,1.44,0.92,1.11,3.31,-1.0,-1.0,457705.0,0,1,0,-1.0,-1.0,-4.854903,-124.173193,-2.600914,15.806897,27.350271,-22.359652,3.3,1.67,-1.0,-1.0,-1.0,84.9,1740.0,6.2,747947,54.26,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,2,0,0,0,1,131.0,-1.0,-1.0,1
1,2024-03-16 00:00:00,96.2,1.78,6.3,"""Skubal, Tarik""",668804,669373,-1,2.0,0,0,1,-1.0,-1,0,0,1.37,1.08,-0.05,2.83,-1.0,-1.0,457705.0,0,1,0,-1.0,-1.0,-7.948987,-139.714018,-5.985534,19.815637,34.920946,-16.847749,3.3,1.67,200.0,82.2,17.0,95.7,2161.0,6.4,747947,54.14,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,1,0,0,0,1,145.0,-1.0,-1.0,0


In [119]:
# [ 'run_diff', 'base_state']


df = df.with_columns(
    (pl.col("fld_score") - pl.col("bat_score")).alias("run_diff").cast(pl.Int32),
)

df = df.drop(["fld_score", "bat_score"])
df.columns


['pitch_type',
 'game_date',
 'release_speed',
 'release_pos_x',
 'release_pos_z',
 'player_name',
 'batter',
 'pitcher',
 'events',
 'zone',
 'stand',
 'p_throws',
 'type',
 'hit_location',
 'bb_type',
 'balls',
 'strikes',
 'pfx_x',
 'pfx_z',
 'plate_x',
 'plate_z',
 'on_3b',
 'on_2b',
 'on_1b',
 'outs_when_up',
 'inning',
 'inning_topbot',
 'hc_x',
 'hc_y',
 'vx0',
 'vy0',
 'vz0',
 'ax',
 'ay',
 'az',
 'sz_top',
 'sz_bot',
 'hit_distance_sc',
 'launch_speed',
 'launch_angle',
 'effective_speed',
 'release_spin_rate',
 'release_extension',
 'game_pk',
 'release_pos_y',
 'estimated_ba_using_speedangle',
 'estimated_woba_using_speedangle',
 'woba_value',
 'woba_denom',
 'babip_value',
 'iso_value',
 'launch_speed_angle',
 'at_bat_number',
 'pitch_number',
 'if_fielding_alignment',
 'of_fielding_alignment',
 'spin_axis',
 'bat_speed',
 'swing_length',
 'next_pitch',
 'run_diff']

In [120]:
df = df.with_columns(
    pl.col("on_1b")
    .map_elements(lambda s: 0 if s == -1.0 else 1, return_dtype=pl.Int32)
    .alias("on_1b"),
    pl.col("on_2b")
    .map_elements(lambda s: 0 if s == -1.0 else 1, return_dtype=pl.Int32)
    .alias("on_2b"),
    pl.col("on_3b")
    .map_elements(lambda s: 0 if s == -1.0 else 1, return_dtype=pl.Int32)
    .alias("on_3b"),
)

df = df.with_columns(
    (pl.col("on_1b") * 3 + pl.col("on_2b") * 5 + pl.col("on_3b") * 7).alias(
        "base_state"
    )
)

df = df.drop(["on_1b", "on_2b", "on_3b"])
df

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,zone,stand,p_throws,type,hit_location,bb_type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,if_fielding_alignment,of_fielding_alignment,spin_axis,bat_speed,swing_length,next_pitch,run_diff,base_state
i64,datetime[μs],f64,f64,f64,str,i64,i64,i64,f64,i64,i64,i64,f64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,i64,i32,i32
7,2024-10-20 00:00:00,78.8,4.1,5.27,"""Manaea, Sean""",571970,640455,4,14.0,1,0,2,-1.0,-1,3,1,-1.04,0.07,1.44,1.65,0,3,0,-1.0,-1.0,-3.876776,-114.790473,-1.213129,-8.519711,20.865706,-31.531093,2.99,1.5,-1.0,-1.0,-1.0,79.7,2141.0,6.8,775302,53.72,-1.0,0.689131,0.7,1.0,0.0,0.0,-1.0,31,5,2,0,294.0,-1.0,-1.0,0,-3,0
0,2024-10-20 00:00:00,83.9,4.16,5.13,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,2,1,1.28,-0.01,-1.13,1.32,0,3,0,-1.0,-1.0,-14.910726,-121.300964,-2.306161,15.532788,23.944392,-32.030777,2.99,1.4,-1.0,-1.0,-1.0,84.7,1835.0,7.2,775302,53.31,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,4,2,0,106.0,-1.0,-1.0,1,-3,0
1,2024-10-20 00:00:00,91.9,4.03,4.97,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,1,1,1.16,0.76,-0.93,2.17,0,3,0,-1.0,-1.0,-15.210428,-132.944842,-2.837582,16.955867,27.981449,-22.734943,2.99,1.53,-1.0,-1.0,-1.0,92.7,2131.0,7.0,775302,53.48,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,3,2,0,113.0,-1.0,-1.0,7,-3,0
7,2024-10-20 00:00:00,80.2,3.84,5.31,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,0,1,-1.34,0.15,-0.86,1.28,0,3,0,-1.0,-1.0,-7.987162,-116.480556,-2.438514,-10.452965,23.959752,-30.532682,2.99,1.35,-1.0,-1.0,-1.0,81.3,2117.0,7.4,775302,53.14,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,2,2,0,284.0,-1.0,-1.0,7,-3,0
7,2024-10-20 00:00:00,78.9,3.89,5.26,"""Manaea, Sean""",571970,640455,-1,8.0,1,0,1,-1.0,-1,0,0,-1.57,0.19,-0.23,1.53,0,3,0,-1.0,-1.0,-6.125394,-114.782271,-1.669025,-12.671426,21.628193,-30.380521,2.99,1.48,-1.0,-1.0,-1.0,79.9,2152.0,7.0,775302,53.45,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,1,2,0,283.0,-1.0,-1.0,2,-3,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,2024-03-16 00:00:00,84.0,1.71,6.4,"""Skubal, Tarik""",668804,669373,-1,14.0,0,0,1,-1.0,-1,1,1,1.52,0.6,1.09,2.08,0,1,0,-1.0,-1.0,-4.432459,-122.07474,-4.703139,16.032842,26.558863,-25.453648,3.3,1.67,-1.0,-1.0,-1.0,83.5,1505.0,6.3,747947,54.23,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,3,0,1,130.0,-1.0,-1.0,0,0,3
0,2024-03-16 00:00:00,85.4,1.95,6.33,"""Skubal, Tarik""",668804,669373,-1,12.0,0,0,2,-1.0,-1,0,1,1.44,0.92,1.11,3.31,0,1,0,-1.0,-1.0,-4.854903,-124.173193,-2.600914,15.806897,27.350271,-22.359652,3.3,1.67,-1.0,-1.0,-1.0,84.9,1740.0,6.2,747947,54.26,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,2,0,1,131.0,-1.0,-1.0,1,0,3
1,2024-03-16 00:00:00,96.2,1.78,6.3,"""Skubal, Tarik""",668804,669373,-1,2.0,0,0,1,-1.0,-1,0,0,1.37,1.08,-0.05,2.83,0,1,0,-1.0,-1.0,-7.948987,-139.714018,-5.985534,19.815637,34.920946,-16.847749,3.3,1.67,200.0,82.2,17.0,95.7,2161.0,6.4,747947,54.14,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,1,0,1,145.0,-1.0,-1.0,0,0,3


In [121]:
df = df.with_columns(
    pl.col("pitch_type")
    .cum_count(reverse=False)
    .over(["player_name", "game_date", "game_pk"])
    .alias("pitches_thrown_curr_game")
)
df = df.sort(
    [
        "game_date",
        "game_pk",
        "at_bat_number",
        "pitch_number",
    ],
    descending=True,
)
df

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,zone,stand,p_throws,type,hit_location,bb_type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,if_fielding_alignment,of_fielding_alignment,spin_axis,bat_speed,swing_length,next_pitch,run_diff,base_state,pitches_thrown_curr_game
i64,datetime[μs],f64,f64,f64,str,i64,i64,i64,f64,i64,i64,i64,f64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,i64,i32,i32,u32
7,2024-10-20 00:00:00,78.8,4.1,5.27,"""Manaea, Sean""",571970,640455,4,14.0,1,0,2,-1.0,-1,3,1,-1.04,0.07,1.44,1.65,0,3,0,-1.0,-1.0,-3.876776,-114.790473,-1.213129,-8.519711,20.865706,-31.531093,2.99,1.5,-1.0,-1.0,-1.0,79.7,2141.0,6.8,775302,53.72,-1.0,0.689131,0.7,1.0,0.0,0.0,-1.0,31,5,2,0,294.0,-1.0,-1.0,0,-3,0,1
0,2024-10-20 00:00:00,83.9,4.16,5.13,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,2,1,1.28,-0.01,-1.13,1.32,0,3,0,-1.0,-1.0,-14.910726,-121.300964,-2.306161,15.532788,23.944392,-32.030777,2.99,1.4,-1.0,-1.0,-1.0,84.7,1835.0,7.2,775302,53.31,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,4,2,0,106.0,-1.0,-1.0,1,-3,0,2
1,2024-10-20 00:00:00,91.9,4.03,4.97,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,1,1,1.16,0.76,-0.93,2.17,0,3,0,-1.0,-1.0,-15.210428,-132.944842,-2.837582,16.955867,27.981449,-22.734943,2.99,1.53,-1.0,-1.0,-1.0,92.7,2131.0,7.0,775302,53.48,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,3,2,0,113.0,-1.0,-1.0,7,-3,0,3
7,2024-10-20 00:00:00,80.2,3.84,5.31,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,0,1,-1.34,0.15,-0.86,1.28,0,3,0,-1.0,-1.0,-7.987162,-116.480556,-2.438514,-10.452965,23.959752,-30.532682,2.99,1.35,-1.0,-1.0,-1.0,81.3,2117.0,7.4,775302,53.14,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,2,2,0,284.0,-1.0,-1.0,7,-3,0,4
7,2024-10-20 00:00:00,78.9,3.89,5.26,"""Manaea, Sean""",571970,640455,-1,8.0,1,0,1,-1.0,-1,0,0,-1.57,0.19,-0.23,1.53,0,3,0,-1.0,-1.0,-6.125394,-114.782271,-1.669025,-12.671426,21.628193,-30.380521,2.99,1.48,-1.0,-1.0,-1.0,79.9,2152.0,7.0,775302,53.45,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,1,2,0,283.0,-1.0,-1.0,2,-3,0,5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,2024-03-16 00:00:00,84.0,1.71,6.4,"""Skubal, Tarik""",668804,669373,-1,14.0,0,0,1,-1.0,-1,1,1,1.52,0.6,1.09,2.08,0,1,0,-1.0,-1.0,-4.432459,-122.07474,-4.703139,16.032842,26.558863,-25.453648,3.3,1.67,-1.0,-1.0,-1.0,83.5,1505.0,6.3,747947,54.23,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,3,0,1,130.0,-1.0,-1.0,0,0,3,68
0,2024-03-16 00:00:00,85.4,1.95,6.33,"""Skubal, Tarik""",668804,669373,-1,12.0,0,0,2,-1.0,-1,0,1,1.44,0.92,1.11,3.31,0,1,0,-1.0,-1.0,-4.854903,-124.173193,-2.600914,15.806897,27.350271,-22.359652,3.3,1.67,-1.0,-1.0,-1.0,84.9,1740.0,6.2,747947,54.26,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,2,0,1,131.0,-1.0,-1.0,1,0,3,69
1,2024-03-16 00:00:00,96.2,1.78,6.3,"""Skubal, Tarik""",668804,669373,-1,2.0,0,0,1,-1.0,-1,0,0,1.37,1.08,-0.05,2.83,0,1,0,-1.0,-1.0,-7.948987,-139.714018,-5.985534,19.815637,34.920946,-16.847749,3.3,1.67,200.0,82.2,17.0,95.7,2161.0,6.4,747947,54.14,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,1,0,1,145.0,-1.0,-1.0,0,0,3,70


In [122]:
import pybaseball as pb

In [123]:
with open("../params.yaml", "r") as file:
    params = yaml.safe_load(file)

In [136]:
# add batter statistics to the dataframe


batting_df = pb.batting_stats_bref(params["clean"]["start_year"])
print(batting_df.columns)
player_ids = list(df.select("batter").unique().to_pandas()["batter"])
batting_df = pl.DataFrame(batting_df[batting_df["mlbID"].isin(player_ids)])

# player_ids

Index(['Name', 'Age', '#days', 'Lev', 'Tm', 'G', 'PA', 'AB', 'R', 'H', '2B',
       '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB',
       'CS', 'BA', 'OBP', 'SLG', 'OPS', 'mlbID'],
      dtype='object')


In [ ]:
batting_df = pb.batting_stats_bref(params["clean"]["start_year"])
print(batting_df.columns)
player_ids = list(df.select("batter").unique().to_pandas()["batter"])
batting_df = pl.DataFrame(batting_df[batting_df["mlbID"].isin(player_ids)])
batting_df = batting_df.drop(
    [
        "Name",
        "Age",
        "#days",
        "Lev",
        "Tm",
        "G",
        "PA",
        "AB",
        "R",
        "H",
        "2B",
        "3B",
        "HR",
        "RBI",
        "BB",
        "IBB",
        "SO",
        "HBP",
        "SH",
        "SF",
        "GDP",
        "SB",
        "CS",
    ]
)
df = df.join(batting_df, left_on="batter", right_on="mlbID", how="left")

['pitch_type', 'game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 'player_name', 'batter', 'pitcher', 'events', 'zone', 'stand', 'p_throws', 'type', 'hit_location', 'bb_type', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk', 'release_pos_y', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 'babip_value', 'iso_value', 'launch_speed_angle', 'at_bat_number', 'pitch_number', 'if_fielding_alignment', 'of_fielding_alignment', 'spin_axis', 'bat_speed', 'swing_length', 'next_pitch', 'run_diff', 'base_state', 'pitches_thrown_curr_game']
['Name', 'Age', '#days', 'Lev', 'Tm', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS', 'BA', '

In [141]:
# Inspect the columns of batting_df
print(batting_df.columns)

# Perform a left join on the 'batter' column
df = df.join(batting_df, left_on="batter", right_on="mlbID", how="left")

['BA', 'OBP', 'SLG', 'OPS', 'mlbID']


In [142]:
df

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,zone,stand,p_throws,type,hit_location,bb_type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,if_fielding_alignment,of_fielding_alignment,spin_axis,bat_speed,swing_length,next_pitch,run_diff,base_state,pitches_thrown_curr_game,BA,OBP,SLG,OPS
i64,datetime[μs],f64,f64,f64,str,i64,i64,i64,f64,i64,i64,i64,f64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,i64,i32,i32,u32,f64,f64,f64,f64
7,2024-10-20 00:00:00,78.8,4.1,5.27,"""Manaea, Sean""",571970,640455,4,14.0,1,0,2,-1.0,-1,3,1,-1.04,0.07,1.44,1.65,0,3,0,-1.0,-1.0,-3.876776,-114.790473,-1.213129,-8.519711,20.865706,-31.531093,2.99,1.5,-1.0,-1.0,-1.0,79.7,2141.0,6.8,775302,53.72,-1.0,0.689131,0.7,1.0,0.0,0.0,-1.0,31,5,2,0,294.0,-1.0,-1.0,0,-3,0,1,0.22,0.364,0.472,0.836
0,2024-10-20 00:00:00,83.9,4.16,5.13,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,2,1,1.28,-0.01,-1.13,1.32,0,3,0,-1.0,-1.0,-14.910726,-121.300964,-2.306161,15.532788,23.944392,-32.030777,2.99,1.4,-1.0,-1.0,-1.0,84.7,1835.0,7.2,775302,53.31,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,4,2,0,106.0,-1.0,-1.0,1,-3,0,2,0.22,0.364,0.472,0.836
1,2024-10-20 00:00:00,91.9,4.03,4.97,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,1,1,1.16,0.76,-0.93,2.17,0,3,0,-1.0,-1.0,-15.210428,-132.944842,-2.837582,16.955867,27.981449,-22.734943,2.99,1.53,-1.0,-1.0,-1.0,92.7,2131.0,7.0,775302,53.48,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,3,2,0,113.0,-1.0,-1.0,7,-3,0,3,0.22,0.364,0.472,0.836
7,2024-10-20 00:00:00,80.2,3.84,5.31,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,0,1,-1.34,0.15,-0.86,1.28,0,3,0,-1.0,-1.0,-7.987162,-116.480556,-2.438514,-10.452965,23.959752,-30.532682,2.99,1.35,-1.0,-1.0,-1.0,81.3,2117.0,7.4,775302,53.14,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,2,2,0,284.0,-1.0,-1.0,7,-3,0,4,0.22,0.364,0.472,0.836
7,2024-10-20 00:00:00,78.9,3.89,5.26,"""Manaea, Sean""",571970,640455,-1,8.0,1,0,1,-1.0,-1,0,0,-1.57,0.19,-0.23,1.53,0,3,0,-1.0,-1.0,-6.125394,-114.782271,-1.669025,-12.671426,21.628193,-30.380521,2.99,1.48,-1.0,-1.0,-1.0,79.9,2152.0,7.0,775302,53.45,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,1,2,0,283.0,-1.0,-1.0,2,-3,0,5,0.22,0.364,0.472,0.836
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,2024-03-16 00:00:00,84.0,1.71,6.4,"""Skubal, Tarik""",668804,669373,-1,14.0,0,0,1,-1.0,-1,1,1,1.52,0.6,1.09,2.08,0,1,0,-1.0,-1.0,-4.432459,-122.07474,-4.703139,16.032842,26.558863,-25.453648,3.3,1.67,-1.0,-1.0,-1.0,83.5,1505.0,6.3,747947,54.23,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,3,0,1,130.0,-1.0,-1.0,0,0,3,68,0.275,0.344,0.447,0.791
0,2024-03-16 00:00:00,85.4,1.95,6.33,"""Skubal, Tarik""",668804,669373,-1,12.0,0,0,2,-1.0,-1,0,1,1.44,0.92,1.11,3.31,0,1,0,-1.0,-1.0,-4.854903,-124.173193,-2.600914,15.806897,27.350271,-22.359652,3.3,1.67,-1.0,-1.0,-1.0,84.9,1740.0,6.2,747947,54.26,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,2,0,1,131.0,-1.0,-1.0,1,0,3,69,0.275,0.344,0.447,0.791
1,2024-03-16 00:00:00,96.2,1.78,6.3,"""Skubal, Tarik""",668804,669373,-1,2.0,0,0,1,-1.0,-1,0,0,1.37,1.08,-0.05,2.83,0,1,0,-1.0,-1.0,-7.948987,-139.714018,-5.985534,19.815637,34.920946,-16.847749,3.3,1.67,200.0,82.2,17.0,95.7,2161.0,6.4,747947,54.14,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,1,0,1,145.0,-1.0,-1.0,0,0,3,70,0.275,0.344,0.447,0.791
